# Lucy Edit Dev - Video Editing with Gradio Interface

This notebook creates a web interface for the Lucy-Edit-Dev model - an open-source instruction-guided video editing model that can perform various edits like clothing changes, character replacements, object insertions, and scene transformations.

## Installation
Install required dependencies

In [1]:
!pip install git+https://github.com/huggingface/diffusers
!pip install gradio torch torchvision torchaudio
!pip install pillow opencv-python imageio-ffmpeg
!pip install accelerate transformers

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-r7cepwri
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-r7cepwri
  Resolved https://github.com/huggingface/diffusers to commit d8310a8fca812ff4afdcc1fd09c135dcccaab670
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.36.0.dev0-py3-none-any.whl size=4247694 sha256=c5a975c6307dc5e9039276dd5c74f1d0724c140733776dc8f46c7162bca37b8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-ujmrrzgz/wheels/90/d4/44/a58bc00fb405fefb633b0d9d2307f6e3aec6cc1775d82555d3
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.35.1
    Uninstalling diffusers-0.35.1:
      Successfully uninstalled diffusers-0.35.1


## Import Libraries

In [2]:
import gradio as gr
import torch
import tempfile
import os
from typing import List, Optional
from PIL import Image
import cv2
import numpy as np
from diffusers import AutoencoderKLWan, LucyEditPipeline
from diffusers.utils import export_to_video, load_video
import gc

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


## Model Loading and Utilities

In [3]:
class LucyEditInterface:
    def __init__(self):
        self.pipe = None
        self.vae = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_id = "decart-ai/Lucy-Edit-Dev"

    def load_model(self):
        """Load the Lucy Edit model"""
        if self.pipe is None:
            try:
                print("Loading Lucy Edit model...")
                self.vae = AutoencoderKLWan.from_pretrained(
                    self.model_id,
                    subfolder="vae",
                    torch_dtype=torch.float32
                )
                self.pipe = LucyEditPipeline.from_pretrained(
                    self.model_id,
                    vae=self.vae,
                    torch_dtype=torch.bfloat16 if self.device == "cuda" else torch.float32
                )
                self.pipe.to(self.device)
                print(f"Model loaded successfully on {self.device}")
            except Exception as e:
                print(f"Error loading model: {e}")
                return False
        return True

    def preprocess_video(self, video_path: str, num_frames: int = 81, width: int = 832, height: int = 480):
        """Preprocess video for the model"""
        try:
            def convert_video(video: List[Image.Image]) -> List[Image.Image]:
                processed = []
                for i, frame in enumerate(video[:num_frames]):
                    resized_frame = frame.resize((width, height))
                    processed.append(resized_frame)
                return processed

            video = load_video(video_path, convert_method=convert_video)
            return video
        except Exception as e:
            print(f"Error preprocessing video: {e}")
            return None

    def edit_video(
        self,
        video_path: str,
        prompt: str,
        negative_prompt: str = "",
        num_frames: int = 81,
        width: int = 832,
        height: int = 480,
        guidance_scale: float = 5.0
    ):
        """Main video editing function"""
        if not self.load_model():
            return None, "Failed to load model"

        if not video_path or not prompt.strip():
            return None, "Please provide both video and prompt"

        try:
            print("Preprocessing video...")
            video_frames = self.preprocess_video(video_path, num_frames, width, height)
            if video_frames is None:
                return None, "Failed to preprocess video"

            print(f"Processing {len(video_frames)} frames with prompt: {prompt}")

            with torch.no_grad():
                output = self.pipe(
                    prompt=prompt,
                    video=video_frames,
                    negative_prompt=negative_prompt,
                    height=height,
                    width=width,
                    num_frames=num_frames,
                    guidance_scale=guidance_scale
                ).frames[0]

            with tempfile.NamedTemporaryFile(suffix=".mp4", delete=False) as tmp_file:
                output_path = tmp_file.name
                export_to_video(output, output_path, fps=24)

            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

            return output_path, "Video edited successfully!"

        except Exception as e:
            error_msg = f"Error during video editing: {str(e)}"
            print(error_msg)
            return None, error_msg

lucy_interface = LucyEditInterface()

## Gradio Interface

In [4]:
def create_gradio_interface():
    """Create the Gradio web interface"""

    example_prompts = {
        "Clothing Change": "Change the shirt to a kimono with wide sleeves and patterned fabric, traditional Japanese design with floral motifs",
        "Character Replacement": "Replace the person with a tiger, striped orange fur, muscular build, and glowing green eyes, realistic animal features",
        "Object Replacement": "Replace the apple with a glowing crystal ball emitting blue light, transparent glass surface with magical glow",
        "Scene Transformation": "Transform the scene into a snowy winter landscape with falling snowflakes, cold atmosphere, white ground",
        "Fantasy Style": "Transform the person into a medieval knight with full armor, sword and shield, heroic pose",
        "Color Change": "Change the jacket color to deep red leather with a glossy finish, rich burgundy tone"
    }

    def process_video(video, prompt, negative_prompt, num_frames, width, height, guidance_scale):
        """Process video with error handling for Gradio"""
        if video is None:
            return None, "Please upload a video file"

        result_video, message = lucy_interface.edit_video(
            video_path=video,
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_frames=int(num_frames),
            width=int(width),
            height=int(height),
            guidance_scale=float(guidance_scale)
        )

        return result_video, message

    def update_prompt(edit_type):
        """Update prompt based on selected edit type"""
        return example_prompts.get(edit_type, "")

    with gr.Blocks(title="Lucy Edit Dev - Video Editing", theme=gr.themes.Soft()) as demo:
        gr.Markdown(
            """
            # 🎬 Lucy Edit Dev - AI Video Editing

            Transform your videos with text instructions! Lucy Edit is an open-source model that can:
            - 👕 Change clothing and accessories
            - 🧌 Replace characters with animals, monsters, or other people
            - 🎨 Transform scenes and backgrounds
            - 🎯 Add or replace objects

            **Tips for best results:**
            - Use 20-30 descriptive words in your prompt
            - Start with trigger words: "Change", "Replace", "Add", "Transform to"
            - Be specific about details, colors, and styles
            - 81 frames work best for temporal consistency
            """
        )

        with gr.Row():
            with gr.Column(scale=1):
                video_input = gr.Video(
                    label="📹 Upload Video",
                    format="mp4"
                )

                edit_type = gr.Dropdown(
                    choices=list(example_prompts.keys()),
                    label="🎨 Edit Type (Optional)",
                    value=None,
                    info="Select for example prompts"
                )

                prompt_input = gr.Textbox(
                    label="✍️ Edit Prompt",
                    placeholder="Describe how you want to edit the video...",
                    lines=3,
                    info="Be detailed and specific for best results"
                )

                negative_prompt_input = gr.Textbox(
                    label="🚫 Negative Prompt (Optional)",
                    placeholder="Things you don't want in the video...",
                    lines=2
                )

                with gr.Accordion("⚙️ Advanced Settings", open=False):
                    num_frames = gr.Slider(
                        minimum=16,
                        maximum=81,
                        value=81,
                        step=1,
                        label="Number of Frames",
                        info="81 frames recommended for best quality"
                    )

                    with gr.Row():
                        width = gr.Slider(
                            minimum=256,
                            maximum=1024,
                            value=832,
                            step=64,
                            label="Width"
                        )
                        height = gr.Slider(
                            minimum=256,
                            maximum=1024,
                            value=480,
                            step=64,
                            label="Height"
                        )

                    guidance_scale = gr.Slider(
                        minimum=1.0,
                        maximum=15.0,
                        value=5.0,
                        step=0.5,
                        label="Guidance Scale",
                        info="Higher values = stronger prompt adherence"
                    )

                process_btn = gr.Button(
                    "🎬 Edit Video",
                    variant="primary",
                    size="lg"
                )

            with gr.Column(scale=1):
                video_output = gr.Video(
                    label="📽️ Edited Video",
                    format="mp4"
                )

                status_output = gr.Textbox(
                    label="Status",
                    interactive=False
                )

        with gr.Accordion("📋 Supported Edit Types", open=False):
            gr.Markdown(
                """
                ### ✅ Best Performance:
                - **Clothing Changes**: Swap outfits while preserving motion and identity
                - **Character Replacement**: Transform people into creatures or other characters
                - **Object Replacement**: Replace objects with similar-sized alternatives

                ### ⚠️ Mixed Results:
                - **Color Changes**: Sometimes subtle, works best with precise descriptions
                - **Add Objects**: Often attaches to subject, best for wearable/handheld items
                - **Scene Transformations**: Effective for backgrounds, might alter subject

                ### 🎯 Trigger Words:
                - **Change** → Clothing or color modifications
                - **Add** → Adding animals or objects
                - **Replace** → Object substitution or subject swap
                - **Transform to** → Global scene or style transformations
                """
            )

        gr.Examples(
            examples=[
                ["Change the shirt to a medieval knight armor with metallic silver finish, detailed chainmail texture", "", 81, 832, 480, 5.0],
                ["Replace the person with a majestic lion, golden mane, powerful stance, realistic fur texture", "", 81, 832, 480, 5.0],
                ["Transform the scene into a cyberpunk cityscape with neon lights, futuristic buildings, dark atmosphere", "", 81, 832, 480, 5.0],
                ["Change the dress to a flowing red evening gown with elegant draping, luxurious fabric", "low quality, blurry", 81, 832, 480, 5.0]
            ],
            inputs=[prompt_input, negative_prompt_input, num_frames, width, height, guidance_scale],
            label="Example Prompts"
        )

        edit_type.change(
            fn=update_prompt,
            inputs=[edit_type],
            outputs=[prompt_input]
        )

        process_btn.click(
            fn=process_video,
            inputs=[
                video_input, prompt_input, negative_prompt_input,
                num_frames, width, height, guidance_scale
            ],
            outputs=[video_output, status_output],
            show_progress=True
        )

    return demo

demo = create_gradio_interface()

## Launch the Interface
Run the cell below to start the web interface

In [5]:
# Launch the Gradio interface
if __name__ == "__main__":
    print("🚀 Starting Lucy Edit Dev interface...")
    print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

    demo.launch(
        share=True,  # Set to False if you don't want public sharing
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True,
        debug=True
    )

🚀 Starting Lucy Edit Dev interface...
Device: GPU
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ab6b0907c74e813324.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://ab6b0907c74e813324.gradio.live
